# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods

Import Libriries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


## 1. Scraping and Explore Dataset

Information page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

The dataset is a list of Toronto's zipcodes which includes the boroughs and neighborhood names.

In [3]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Tranform Data

In [12]:
df_toronto.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df_toronto.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()


(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")

latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
latlong.shape

(103, 3)

In [26]:
latlong.tail()

,PostalCode,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [30]:
#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("Postal Code")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong, left_index=True, right_index=True)
neighbor.head()

,Postal Code,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [31]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


In [34]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [35]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [36]:
# Foursquare credentials and version

CLIENT_ID = 'QTXIPLN5DZXFGKB5AORTKHCCCXFDHRE3WBDI0HBWKA1G3BKZ' # your Foursquare ID
CLIENT_SECRET = '52HELZRA5IRZVAADFKBDCCMYTQYSIHEJN5ML0GGRGQWWTX0L' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: QTXIPLN5DZXFGKB5AORTKHCCCXFDHRE3WBDI0HBWKA1G3BKZ


Review East York. Because my friend lives there. 

In [66]:
eastYork_data = neighbor[neighbor['Borough'] == 'East York'].reset_index(drop=True)
eastYork_data.head()

,Postal Code,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937
1,M4C,East York,Woodbine Heights,M4C,43.695344,-79.318389
2,M4G,East York,Leaside,M4G,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,M4H,43.705369,-79.349372
4,M4J,East York,"East Toronto, Broadview North (Old East York)",M4J,43.685347,-79.338106


In [40]:
eastYork_data.shape

(5, 6)

Explore Neighborhoods in San Antonio

In [42]:

address = 'East York, Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East York are 43.699971000000005, -79.33251996261595.


In [43]:
# create map of Scarborough using latitude and longitude values
map_eastYork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(eastYork_data['Latitude'], eastYork_data['Longitude'], eastYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eastYork)  
    
map_eastYork

   ## Explore East York neighborhood in Toronto with Foursquare API

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [45]:
eastYork_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 74 venues in 5 neighborhoods.


In [46]:
print(eastYork_venues.shape)
eastYork_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
3,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
4,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place


In [47]:
#Venues per Neighborhood
eastYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"East Toronto, Broadview North (Old East York)",4,4,4,4,4,4
Leaside,32,32,32,32,32,32
"Parkview Hill, Woodbine Gardens",10,10,10,10,10,10
Thorncliffe Park,22,22,22,22,22,22
Woodbine Heights,6,6,6,6,6,6


In [48]:
print('There are {} distinct venues in {} categories.'.format(
    len(eastYork_venues['Venue'].unique()),len(eastYork_venues['Venue Category'].unique())))

There are 67 distinct venues in 45 categories.


In [49]:
# one hot encoding
eastYork_onehot = pd.get_dummies(eastYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastYork_onehot['Neighborhood'] = eastYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastYork_onehot.columns[-1]] + list(eastYork_onehot.columns[:-1])
eastYork_onehot = eastYork_onehot[fixed_columns]

eastYork_onehot.head()

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Café,Coffee Shop,Convenience Store,Curling Ice,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio
0,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,"Parkview Hill, Woodbine Gardens",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


We can see there is not so much of one Hot in East York. 
Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [50]:
eastYork_grouped = eastYork_onehot.groupby('Neighborhood').mean().reset_index()
eastYork_grouped

,Neighborhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Bus Stop,Café,Coffee Shop,Convenience Store,Curling Ice,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio
0,"East Toronto, Broadview North (Old East York)",0.0,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.00000,0.250000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000
1,Leaside,0.0,0.03125,0.062500,0.031250,0.03125,0.03125,0.03125,0.062500,0.000000,0.000000,0.0,0.093750,0.00,0.000000,0.03125,0.03125,0.000000,0.03125,0.000000,0.03125,0.0625,0.000000,0.0,0.031250,0.000000,0.0,0.000000,0.000000,0.031250,0.03125,0.000000,0.03125,0.000000,0.000000,0.031250,0.031250,0.03125,0.000000,0.03125,0.09375,0.03125,0.031250,0.03125,0.000000,0.000000
2,"Parkview Hill, Woodbine Gardens",0.1,0.00000,0.100000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.1,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.0000,0.000000,0.1,0.000000,0.000000,0.1,0.000000,0.100000,0.000000,0.00000,0.000000,0.10000,0.100000,0.200000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000
3,Thorncliffe Park,0.0,0.00000,0.045455,0.000000,0.00000,0.00000,0.00000,0.045455,0.045455,0.000000,0.0,0.045455,0.00,0.000000,0.00000,0.00000,0.045455,0.00000,0.045455,0.00000,0.0000,0.045455,0.0,0.045455,0.045455,0.0,0.090909,0.045455,0.045455,0.00000,0.045455,0.00000,0.045455,0.045455,0.045455,0.090909,0.00000,0.000000,0.00000,0.00000,0.00000,0.045455,0.00000,0.045455,0.045455
4,Woodbine Heights,0.0,0.00000,0.000000,0.166667,0.00000,0.00000,0.00000,0.000000,0.000000,0.166667,0.0,0.000000,0.00,0.166667,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.166667,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.333333,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000


In [53]:
num_top_venues = 5

for hood in eastYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = eastYork_grouped[eastYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Toronto, Broadview North (Old East York)----
                  venue  freq
0          Intersection  0.50
1     Convenience Store  0.25
2                  Park  0.25
3    Athletics & Sports  0.00
4  Gym / Fitness Center  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2                    Bank  0.06
3            Burger Joint  0.06
4  Furniture / Home Store  0.06


----Parkview Hill, Woodbine Gardens----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2                Café   0.1
3            Pharmacy   0.1
4           Pet Store   0.1


----Thorncliffe Park----
               venue  freq
0     Sandwich Place  0.09
1  Indian Restaurant  0.09
2       Liquor Store  0.05
3                Gym  0.05
4      Grocery Store  0.05


----Woodbine Heights----
          venue  freq
0  Skating Rink  0.33
1    Beer Store  0.17
2          Park  0.17
3      Bus Stop  0.17
4   Curling Ice  0.1

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eastYork_grouped['Neighborhood']

for ind in np.arange(eastYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"East Toronto, Broadview North (Old East York)",Intersection,Park,Convenience Store,Yoga Studio,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Curling Ice,Café,Furniture / Home Store,Bus Stop,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop,Beer Store
1,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Bank,Mexican Restaurant,Bike Shop,Dessert Shop,Electronics Store,Fish & Chips Shop,Grocery Store,Brewery,Breakfast Spot,Liquor Store,Department Store,Pet Store,Beer Store,Restaurant,Sandwich Place,Shopping Mall
2,"Parkview Hill, Woodbine Gardens",Pizza Place,Gastropub,Bank,Café,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports,Sandwich Place,Restaurant,Dessert Shop,Department Store,Curling Ice,Convenience Store,Coffee Shop,Supermarket,Bus Stop,Bus Line,Burger Joint
3,Thorncliffe Park,Sandwich Place,Indian Restaurant,Yoga Studio,Park,Discount Store,Fast Food Restaurant,Coffee Shop,Gas Station,Warehouse Store,Grocery Store,Gym,Intersection,Liquor Store,Bus Line,Burger Joint,Pharmacy,Pizza Place,Restaurant,Bank,Supermarket
4,Woodbine Heights,Skating Rink,Bus Stop,Beer Store,Curling Ice,Park,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Convenience Store,Yoga Studio,Café,Furniture / Home Store,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop


In [73]:
neighborhoods_venues_sorted.iloc[:,10]

0     Department Store
1    Fish & Chips Shop
2       Sandwich Place
3        Grocery Store
4         Dessert Shop
Name: 10th Most Common Venue, dtype: object

In [74]:
# set number of clusters
kclusters = 3

eastYork_grouped_clustering = eastYork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(eastYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 1, 1, 1, 2], dtype=int32)

In [75]:

eastYork_merged = eastYork_data

# add clustering labels
eastYork_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastYork_merged = eastYork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

eastYork_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4B,East York,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937,0,Pizza Place,Gastropub,Bank,Café,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports,Sandwich Place,Restaurant,Dessert Shop,Department Store,Curling Ice,Convenience Store,Coffee Shop,Supermarket,Bus Stop,Bus Line,Burger Joint
1,M4C,East York,Woodbine Heights,M4C,43.695344,-79.318389,1,Skating Rink,Bus Stop,Beer Store,Curling Ice,Park,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Convenience Store,Yoga Studio,Café,Furniture / Home Store,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop
2,M4G,East York,Leaside,M4G,43.709060,-79.363452,1,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Bank,Mexican Restaurant,Bike Shop,Dessert Shop,Electronics Store,Fish & Chips Shop,Grocery Store,Brewery,Breakfast Spot,Liquor Store,Department Store,Pet Store,Beer Store,Restaurant,Sandwich Place,Shopping Mall
3,M4H,East York,Thorncliffe Park,M4H,43.705369,-79.349372,1,Sandwich Place,Indian Restaurant,Yoga Studio,Park,Discount Store,Fast Food Restaurant,Coffee Shop,Gas Station,Warehouse Store,Grocery Store,Gym,Intersection,Liquor Store,Bus Line,Burger Joint,Pharmacy,Pizza Place,Restaurant,Bank,Supermarket
4,M4J,East York,"East Toronto, Broadview North (Old East York)",M4J,43.685347,-79.338106,2,Intersection,Park,Convenience Store,Yoga Studio,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Curling Ice,Café,Furniture / Home Store,Bus Stop,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop,Beer Store


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastYork_merged['Latitude'], eastYork_merged['Longitude'], eastYork_merged['Neighborhood'], eastYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
eY_cluster_0 = eastYork_merged.loc[eastYork_merged['Cluster Labels'] == 0, eastYork_merged.columns[[1] + list(range(4, eastYork_merged.shape[1]))]]

eY_cluster_1 = eastYork_merged.loc[eastYork_merged['Cluster Labels'] == 1, eastYork_merged.columns[[1] + list(range(4, eastYork_merged.shape[1]))]]

eY_cluster_2 = eastYork_merged.loc[eastYork_merged['Cluster Labels'] == 2, eastYork_merged.columns[[1] + list(range(4, eastYork_merged.shape[1]))]]

In [79]:
eY_cluster_0

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,East York,43.706397,-79.309937,0,Pizza Place,Gastropub,Bank,Café,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Athletics & Sports,Sandwich Place,Restaurant,Dessert Shop,Department Store,Curling Ice,Convenience Store,Coffee Shop,Supermarket,Bus Stop,Bus Line,Burger Joint


In [81]:
eY_cluster_1

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,East York,43.695344,-79.318389,1,Skating Rink,Bus Stop,Beer Store,Curling Ice,Park,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Convenience Store,Yoga Studio,Café,Furniture / Home Store,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop
2,East York,43.709060,-79.363452,1,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Bank,Mexican Restaurant,Bike Shop,Dessert Shop,Electronics Store,Fish & Chips Shop,Grocery Store,Brewery,Breakfast Spot,Liquor Store,Department Store,Pet Store,Beer Store,Restaurant,Sandwich Place,Shopping Mall
3,East York,43.705369,-79.349372,1,Sandwich Place,Indian Restaurant,Yoga Studio,Park,Discount Store,Fast Food Restaurant,Coffee Shop,Gas Station,Warehouse Store,Grocery Store,Gym,Intersection,Liquor Store,Bus Line,Burger Joint,Pharmacy,Pizza Place,Restaurant,Bank,Supermarket


In [82]:
eY_cluster_2

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,East York,43.685347,-79.338106,2,Intersection,Park,Convenience Store,Yoga Studio,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Dessert Shop,Department Store,Curling Ice,Café,Furniture / Home Store,Bus Stop,Bus Line,Burger Joint,Brewery,Breakfast Spot,Bike Shop,Beer Store


Analizing this venue, I think taht if I am going to live in Toronto with one of my best friends, will be in the cluster number 2, there are more venues and also a Mexican Restaurant! I love it 

## Explore New York City Neighborhoods

In [84]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [85]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [86]:
neighborhoods_data = newyork_data['features']

In [87]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [88]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [89]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [90]:

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [91]:
neighborhoods.shape

(306, 4)

In [92]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [94]:

address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [95]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [97]:
b_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
b_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


I always wanted to live in Brroklyn

In [99]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent = 'foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [100]:
# create map of Manhattan using latitude and longitude values
map_b = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(b_data['Latitude'], b_data['Longitude'], b_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_b)  
    
map_b

In [101]:
b_data.loc[10, 'Neighborhood']

'East Flatbush'

In [102]:

#East Flatbush Latitude and Longitude values

neighborhood_latitude = b_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = b_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = b_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of East Flatbush are 40.64171776668961, -73.93610256185836.


In [103]:
# type your answer here
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=QTXIPLN5DZXFGKB5AORTKHCCCXFDHRE3WBDI0HBWKA1G3BKZ&client_secret=52HELZRA5IRZVAADFKBDCCMYTQYSIHEJN5ML0GGRGQWWTX0L&v=20180604&ll=40.64171776668961,-73.93610256185836&radius=500&limit=50'

In [104]:
#Send the GET request
results = requests.get(url).json()

In [105]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [106]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,VIVID Caribbean American Bistro,Caribbean Restaurant,40.642025,-73.932636
1,Paerdegat Park,Park,40.638137,-73.938138
2,Rite Aid,Pharmacy,40.641799,-73.937224
3,Key Food,Supermarket,40.641806,-73.936490
4,Kennedy Fried Chicken,Fast Food Restaurant,40.641409,-73.937811


In [107]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [108]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [109]:
b_venues = getNearbyVenues(names=b_data['Neighborhood'],
                                   latitudes=b_data['Latitude'],
                                   longitudes=b_data['Longitude'])

Found 2127 venues in 70 neighborhoods.


In [110]:
b_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place


In [111]:

print(b_venues.shape)
b_venues.tail()

(2127, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2122,Erasmus,40.646926,-73.948177,Corner Store,40.648738,-73.944831,Convenience Store
2123,Erasmus,40.646926,-73.948177,P.S. 181,40.648216,-73.946693,School
2124,Erasmus,40.646926,-73.948177,Katherin's Kettle Korn,40.645851,-73.946226,Food Truck
2125,Erasmus,40.646926,-73.948177,ideal Fresh Market,40.647974,-73.943826,Grocery Store
2126,Erasmus,40.646926,-73.948177,Ambrosia Health Foods,40.650760,-73.946333,Health Food Store


In [112]:
#Venues per Neighborhood
b_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,47,47,47,47,47,47
Bay Ridge,50,50,50,50,50,50
Bedford Stuyvesant,26,26,26,26,26,26
Bensonhurst,32,32,32,32,32,32
Bergen Beach,6,6,6,6,6,6
Boerum Hill,50,50,50,50,50,50
Borough Park,20,20,20,20,20,20
Brighton Beach,44,44,44,44,44,44
Broadway Junction,14,14,14,14,14,14


In [113]:
print('There are {} distinct venues in {} categories.'.format(
    len(b_venues['Venue'].unique()),len(b_venues['Venue Category'].unique())))

There are 1777 distinct venues in 264 categories.


In [114]:
# one hot encoding
b_onehot = pd.get_dummies(b_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
b_onehot['Neighborhood'] = b_venues['Neighborhood'] 

neighbor = b_onehot['Neighborhood']
b_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
b_onehot.insert(0, 'Neighborhood', neighbor)

b_onehot.head()

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [115]:

b_grouped = b_onehot.groupby('Neighborhood').mean().reset_index()
b_grouped

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bath Beach,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.021277,0.000000,0.000000,0.021277,0.021277,0.021277,0.000000,0.000000,0.000,0.000000

In [118]:
num_top_venues = 5

for hood in b_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = b_grouped[b_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0            Pharmacy  0.06
1  Chinese Restaurant  0.04
2  Italian Restaurant  0.04
3          Donut Shop  0.04
4         Pizza Place  0.04


----Bay Ridge----
                venue  freq
0                 Spa  0.10
1  Italian Restaurant  0.08
2         Pizza Place  0.06
3    Greek Restaurant  0.04
4          Hookah Bar  0.04


----Bedford Stuyvesant----
         venue  freq
0  Coffee Shop  0.12
1          Bar  0.08
2  Pizza Place  0.08
3         Café  0.08
4    Juice Bar  0.04


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.16
1          Donut Shop  0.06
2       Grocery Store  0.06
3  Italian Restaurant  0.06
4      Ice Cream Shop  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1          Playground  0.17
2      Baseball Field  0.17
3        Hockey Field  0.17
4  Athletics & Sports  0.17


----Boerum Hill----
                    venue  freq
0                     Bar  0.08
1  

In [119]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [121]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = b_grouped['Neighborhood']

for ind in np.arange(b_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(b_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Pizza Place,Italian Restaurant,Fast Food Restaurant,Gas Station,Donut Shop,Sushi Restaurant,Bubble Tea Shop,Chinese Restaurant,Russian Restaurant
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Hookah Bar,Ice Cream Shop,Chinese Restaurant,Grocery Store,Bagel Shop,Greek Restaurant
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,Bagel Shop,New American Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Boutique,Gift Shop
3,Bensonhurst,Chinese Restaurant,Italian Restaurant,Grocery Store,Bakery,Sushi Restaurant,Donut Shop,Ice Cream Shop,Park,Cosmetics Shop,Cha Chaan Teng
4,Bergen Beach,Harbor / Marina,Baseball Field,Athletics & Sports,Playground,Hockey Field,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
5,Boerum Hill,Bar,Furniture / Home Store,Coffee Shop,Yoga Studio,Sandwich Place,French Restaurant,Spa,Jewelry Store,Burrito Place,Seafood Restaurant
6,Borough Park,Bank,Pharmacy,Fast Food Restaurant,Pizza Place,Coffee Shop,Metro Station,Bakery,Café,Chinese Restaurant,Restaurant
7,Brighton Beach,Restaurant,Beach,Russian Restaurant,Eastern European Restaurant,Mobile Phone Shop,Sushi Restaurant,Gourmet Shop,Bank,Fast Food Restaurant,Non-Profit
8,Broadway Junction,Diner,Donut Shop,Intersection,Gas Station,Sandwich Place,Caribbean Restaurant,Burger Joint,Nightclub,Fried Chicken Joint,Dessert Shop
9,Brooklyn Heights,Yoga Studio,Park,Thai Restaurant,Ice Cream Shop,Gym,Mexican Restaurant,Pet Store,Pizza Place,Coffee Shop,Deli / Bodega


In [133]:
neighborhoods_venues_sorted.iloc[10,]

Neighborhood                      Brownsville
1st Most Common Venue              Restaurant
2nd Most Common Venue     Fried Chicken Joint
3rd Most Common Venue      Chinese Restaurant
4th Most Common Venue              Playground
5th Most Common Venue           Moving Target
6th Most Common Venue                   Plaza
7th Most Common Venue          Farmers Market
8th Most Common Venue      Spanish Restaurant
9th Most Common Venue                    Park
10th Most Common Venue      Convenience Store
Name: 10, dtype: object

In [134]:
# set number of clusters
kclusters = 5

b_grouped_clustering = b_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(b_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([1, 4, 4, 1, 0, 4, 1, 1, 1, 4, 1, 4, 1, 4, 1, 4, 4, 1, 1, 1, 1, 4,
       4, 3, 1, 1, 4, 1, 1, 1, 4, 1, 4, 1, 1, 4, 4, 4, 1, 1, 1, 4, 1, 1,
       1, 1, 1, 2, 1, 4, 1, 1, 1, 4, 4, 1, 1, 4, 1, 1, 4, 1, 4, 1, 1, 4,
       1, 4, 4, 1], dtype=int32)

In [135]:
b_merged = b_data

# add clustering labels
b_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
b_merged = b_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

b_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,1,Spa,Italian Restaurant,Pizza Place,American Restaurant,Hookah Bar,Ice Cream Shop,Chinese Restaurant,Grocery Store,Bagel Shop,Greek Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,4,Chinese Restaurant,Italian Restaurant,Grocery Store,Bakery,Sushi Restaurant,Donut Shop,Ice Cream Shop,Park,Cosmetics Shop,Cha Chaan Teng
2,Brooklyn,Sunset Park,40.645103,-74.010316,4,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Bank,Deli / Bodega,Pharmacy,Fried Chicken Joint,Gym
3,Brooklyn,Greenpoint,40.730201,-73.954241,1,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Yoga Studio,Mexican Restaurant,Café,Record Shop,Restaurant,Beer Store
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Italian Restaurant,Lounge,Bakery,Martial Arts Dojo,Bus Station,Spa,Furniture / Home Store,Breakfast Spot,Men's Store


In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_merged['Latitude'], b_merged['Longitude'], b_merged['Neighborhood'], b_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [137]:
b_cluster_0 = b_merged.loc[b_merged['Cluster Labels'] == 0, b_merged.columns[[1] + list(range(4, b_merged.shape[1]))]]

b_cluster_1 = b_merged.loc[b_merged['Cluster Labels'] == 1, b_merged.columns[[1] + list(range(4, b_merged.shape[1]))]]

b_cluster_2 = b_merged.loc[b_merged['Cluster Labels'] == 2, b_merged.columns[[1] + list(range(4, b_merged.shape[1]))]]

b_cluster_3 = b_merged.loc[b_merged['Cluster Labels'] == 3, b_merged.columns[[1] + list(range(4, b_merged.shape[1]))]]

b_cluster_4 = b_merged.loc[b_merged['Cluster Labels'] == 4, b_merged.columns[[1] + list(range(4, b_merged.shape[1]))]]

In [138]:
b_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Gravesend,0,Pizza Place,Italian Restaurant,Lounge,Bakery,Martial Arts Dojo,Bus Station,Spa,Furniture / Home Store,Breakfast Spot,Men's Store


In [139]:
b_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,1,Spa,Italian Restaurant,Pizza Place,American Restaurant,Hookah Bar,Ice Cream Shop,Chinese Restaurant,Grocery Store,Bagel Shop,Greek Restaurant
3,Greenpoint,1,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Yoga Studio,Mexican Restaurant,Café,Record Shop,Restaurant,Beer Store
6,Sheepshead Bay,1,Dessert Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Grocery Store,Playground,Diner,Creperie,Park,Restaurant
7,Manhattan Terrace,1,Pizza Place,Ice Cream Shop,Donut Shop,Deli / Bodega,Organic Grocery,Liquor Store,Coffee Shop,Steakhouse,Restaurant,Grocery Store
8,Flatbush,1,Mexican Restaurant,Caribbean Restaurant,Deli / Bodega,Coffee Shop,Chinese Restaurant,Juice Bar,Bank,Bagel Shop,Sandwich Place,Lounge
10,East Flatbush,1,Moving Target,Supermarket,Department Store,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Fast Food Restaurant,Food & Drink Shop,Print Shop
12,Windsor Terrace,1,Café,Diner,Plaza,Grocery Store,Deli / Bodega,Park,Butcher,Sushi Restaurant,Bookstore,Salad Place
14,Brownsville,1,Restaurant,Fried Chicken Joint,Chinese Restaurant,Playground,Moving Target,Plaza,Farmers Market,Spanish Restaurant,Park,Convenience Store
17,Bedford Stuyvesant,1,Coffee Shop,Café,Pizza Place,Bar,Bagel Shop,New American Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Boutique,Gift Shop
18,Brooklyn Heights,1,Yoga Studio,Park,Thai Restaurant,Ice Cream Shop,Gym,Mexican Restaurant,Pet Store,Pizza Place,Coffee Shop,Deli / Bodega


In [140]:
b_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Prospect Park South,2,Caribbean Restaurant,Grocery Store,Pizza Place,Mobile Phone Shop,Fast Food Restaurant,Latin American Restaurant,Fried Chicken Joint,Mexican Restaurant,Women's Store,Pharmacy


In [141]:
b_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Fort Greene,3,Italian Restaurant,New American Restaurant,Cocktail Bar,Flower Shop,Wine Bar,Playground,Coffee Shop,French Restaurant,Pizza Place,Opera House


In [142]:
b_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,4,Chinese Restaurant,Italian Restaurant,Grocery Store,Bakery,Sushi Restaurant,Donut Shop,Ice Cream Shop,Park,Cosmetics Shop,Cha Chaan Teng
2,Sunset Park,4,Bakery,Latin American Restaurant,Mexican Restaurant,Pizza Place,Mobile Phone Shop,Bank,Deli / Bodega,Pharmacy,Fried Chicken Joint,Gym
5,Brighton Beach,4,Restaurant,Beach,Russian Restaurant,Eastern European Restaurant,Mobile Phone Shop,Sushi Restaurant,Gourmet Shop,Bank,Fast Food Restaurant,Non-Profit
9,Crown Heights,4,Pizza Place,Café,Museum,Bakery,Fried Chicken Joint,Kosher Restaurant,Pharmacy,Deli / Bodega,Coffee Shop,Salon / Barbershop
11,Kensington,4,Grocery Store,Thai Restaurant,Pizza Place,Ice Cream Shop,Café,Music Venue,Mobile Phone Shop,Nail Salon,Mexican Restaurant,Gas Station
13,Prospect Heights,4,Bar,Thai Restaurant,Cocktail Bar,New American Restaurant,Café,Wine Shop,Ice Cream Shop,Mexican Restaurant,Pizza Place,Coffee Shop
15,Williamsburg,4,Pizza Place,Bagel Shop,Coffee Shop,Bar,Yoga Studio,Tapas Restaurant,Burger Joint,Latin American Restaurant,Café,Event Space
16,Bushwick,4,Bar,Mexican Restaurant,Coffee Shop,Bakery,Discount Store,Pizza Place,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Nightclub,Mediterranean Restaurant
21,Red Hook,4,Seafood Restaurant,Art Gallery,Park,Bar,Flower Shop,Ice Cream Shop,Wine Shop,American Restaurant,Café,Farm
22,Gowanus,4,Italian Restaurant,Bar,Furniture / Home Store,Pizza Place,Coffee Shop,Wine Shop,Art Gallery,Gym / Fitness Center,Cocktail Bar,Food & Drink Shop


It is clear that I would prefer to live in cluster number 1 or 4. 